Due 1/27/17: 

-Find the electron quantum number (nx, ny, m) up to E_nx_ny = 10 hbar w

-Make a small program that sets this table, arbitrary cut in n hbar w (first do n = 10)

-Look up analytical form of psi_nx_ny(x,y)

-Set up table in paper and pencil then make program

In [15]:
import numpy as np
from sympy import hermite, diff
from sympy.abc import x, n
import scipy
from Vector import Vector as vec
import math
import scipy.linalg as la
from decimal import *

def magic(n):
    #given shell number n, returns magic number at n
    result = 0
    for i in range(1,n+1):
        result += 2*i
    return result

def energy(n):
    #given shell number n, returns energy at n
    result = 0
    for i in range(1,n+1):
        result += 2*i**2
    return result

def nx_ny(n):
    #given shell number n, returns tuple of possible quantum numbers in (nx, ny) form
    quantum_numbers = []
    for i in range(n+1):
        quantum_numbers.append(vec(i,n-i))
    return tuple(quantum_numbers)

def n_m(e):
    quantum_numbers = []
    for m in range(e-1,-e,-2):
        quantum_numbers.append(vec((e-abs(m)-1)//2,m))
    #for m in range(-e,e,2):
    #    quantum_numbers.append((()))
    #for m in range(e-1,-1,-2):
        #quantum_numbers.append(((e-abs(m)-1)//2,m))
        #quantum_numbers.append(((e-abs(m)-1)//2,-m))


    return tuple(quantum_numbers)

def wavefunction(n):
    #given shell number n, returns the hermite polynomial at that n
    return hermite(n,x)


class single_particle():

    def __init__(self, i, n, m, m_s, state):
        #takes in n, corresponding to shell number, where n = 1,2,3...
        self.shell = i
        self.n = n
        self.m = m
        self.degeneracy = 2*i
        self.E_nx_ny = str(i)
        self.magic_number = magic(i)
        self.energy = energy(i)
        self.nx_ny = []
        self.n_m = [n,m]
        self.state = state
        self.m_s = m_s
        self.wavefunction = wavefunction(i)
        self.l = abs(self.m)
        self.j = self.l + self.m_s
        #self.j_tot = math.sqrt(self.j*(self.j+1))
        
        if self.m_s < 0:
            self.spin_state = vec(0,1)
        elif self.m_s > 0:
            self.spin_state = vec(1,0)
#         if self.m_s > 0:
#             self.spin_state = vec(1,0)
#         elif self.m_s < 0:
#             self.spin_state = vec(0,1)

    def data_tuple(self):
        return (self.shell, self.degeneracy, self.E_nx_ny, self.magic_number, 
                self.energy, self.nx_ny, self.n_m, self.wavefunction)


In [2]:
#Fill the table

import pandas as pd

s_lst = [-0.5,0.5]
#attempt to generalize fill_arrays rather than hard code
def fill_arrays(n, quantities):
    count = 0

    for i in range(n-1):
        #create list of possible quantum numbers
        #i = quantum number arrays index
        nx_ny_lst = nx_ny(i)
        n_m_lst = n_m(i+1)
        n_m_lst = n_m_lst[::-1]
            

        for j in range(i+1):
            
            for s in s_lst:
                #create instance of particle, each with unique quantum numbers
                n = n_m_lst[j].x
                m = n_m_lst[j].y
                particle = single_particle(i+1,n,m,s,count)
                particle.nx_ny = nx_ny_lst[j]
                particle.n_m = n_m_lst[j]
                #particle.m_s = k
                #particle.state = count
                particles.append(particle)
                data = (particle.shell, particle.degeneracy, particle.E_nx_ny, particle.magic_number, 
                    particle.energy, particle.nx_ny, particle.n_m, particle.wavefunction)
                count += 1

                for l in range(len(quantities)-1):
                    #store quantities in different places;
                    #for data that have type int, such as energy, E_nx_ny, etc. I put them in arrays, inserted in the 'else' block
                    #for data with other types, such as quantum number (tuple), I put them in list, inserted in the 'if' block
                    if isinstance(quantities[l],list):
                        quantities[l].append(data[l])
                    else:
                        quantities[l][i] = data[l]


shells=4
data_dict = {}

shell_arr = np.empty(shells-1)
degeneracy_arr = np.empty(shells-1)
E_nx_ny_arr = np.empty(shells-1)
magic_number_arr = np.empty(shells-1)
energy_arr = np.empty(shells-1) 
nx_ny_arr = []
n_m_arr = []
wavefunction_arr = []
particles = []
quantities = [shell_arr, degeneracy_arr, E_nx_ny_arr, magic_number_arr, energy_arr, nx_ny_arr, n_m_arr, wavefunction_arr]
names = ("Shell", "Degeneracy", "E_nx_ny", "Magic Numbers", "Energy", "(nx, ny)", "(n, m)", "Wavefunctions")


fill_arrays(shells,quantities)





In [17]:
'''write to files'''
'''nucleispnumbers is a file with the values: state, n, m, 1, 2*m_s'''
'''nucletwobody is a file with the values: state 1 (p), state 2 (q), spin state of p * q'''


file_spnumbers = open('nucleispnumbers.dat', 'w')
file_twobody = open('nucleitwobody.dat', 'w')
#header = "|n>,n,m,2s,2m_s"
#header = "shell,n,l,j,m,m_s"
header = "|i>,(n,m), spin state,m_s"
#file.write(header)
#file.write("\n")
print(header)

for p in particles:
    #q_str = "|{}>, {}, {}, {}".format(p.state,p.n_m,1,2*p.m_s)
    s_str = "|{}>, ({},{}), {}, {}".format(p.state,p.n,p.m,p.spin_state, 2*p.m_s)
    #file.write(quantum_no_str)
    #file.write("\n")
    sp_str = "{},{},{},{},{}".format(p.state,p.n,p.m,1,int(2*p.m_s))
    file_spnumbers.write(sp_str)
    file_spnumbers.write("\n")
    for q in particles:
        two_body_str = "{},{},{},".format(p.state,q.state,p.spin_state.dot(q.spin_state))
        file_twobody.write(two_body_str)
        file_twobody.write("\n")
#         for r in particles:

#             for s in particles:
#                 H_0 = p.shell + q.shell + r.shell + s.shell
#                 H = H_0 + 0
#                 two_body_str = "{},{},{},{},{}".format(p.state,q.state,r.state,s.state,H)
#                 file_twobody.write(two_body_str)
#                 file_twobody.write("\n")
#     #print(q_str)
#     #print(quantum_no_str)
    print(s_str)
file_spnumbers.close()
file_twobody.close()


|i>,(n,m), spin state,m_s
|0>, (0,0), (0, 1), -1.0
|1>, (0,0), (1, 0), 1.0
|2>, (0,-1), (0, 1), -1.0
|3>, (0,-1), (1, 0), 1.0
|4>, (0,1), (0, 1), -1.0
|5>, (0,1), (1, 0), 1.0
|6>, (0,-2), (0, 1), -1.0
|7>, (0,-2), (1, 0), 1.0
|8>, (1,0), (0, 1), -1.0
|9>, (1,0), (1, 0), 1.0
|10>, (0,2), (0, 1), -1.0
|11>, (0,2), (1, 0), 1.0
0
1
2
3
4
5
6
7
8
9
10
11


In [4]:
import Coulomb

def single_H(hbar_omega, shell):
    return hbar_omega*(shell+1)

states = []
n = []
m = []
m_s = []
non_zero_combinations = []
NParticles=6
max_iter = 100
epsilon = 10**(-4)
hw=1


with open("nucleispnumbers.dat", "r") as qnumfile:
    for line in qnumfile:
        nums = line.split(",")
        states.append(nums[0])
        n.append(int(nums[1]))
        m.append(int(nums[2]))
        m_s.append(int(nums[4]))

spOrbitals = len(states)
two_interaction = np.zeros([spOrbitals,spOrbitals])

with open("nucleitwobody.dat", "r") as twobodyfile:
    for line in twobodyfile:
        nums = line.split(",")
        '''Matrix two_interaction has row, column indices for the first and second interacting electron respectively'''
        '''Value at those indices is the inner product of the two spin states'''
        two_interaction[int(nums[0])][int(nums[1])] = int(nums[2])

hbar_omega = 1
CMatrix = np.eye(spOrbitals) # HF coefficients
DensityMatrix = np.zeros([spOrbitals,spOrbitals])
HFMatrix = np.zeros([spOrbitals,spOrbitals])
Coulomb.write_coulomb_file(hw,nuclei_file="nucleispnumbers.dat")
CoulombMatrix = Coulomb.read_coulomb_file(spOrbitals,coulomb_file="coulomb.dat")




In [14]:
singleparticleH = np.zeros(spOrbitals)
hf_count = 0
maxHFiter = 100
difference = 1
epsilon = 10**-3
for i in range(spOrbitals):
    singleparticleH[i] = Decimal(single_H(hbar_omega,i))
    
with open("hf_energies.txt", "w") as hffile:
    oldenergies = np.zeros(spOrbitals)
    newenergies = np.zeros(spOrbitals)
    while hf_count < maxHFiter and difference > epsilon:
        for alpha in range(len(particles)):
            #a = particles[alpha]

            for beta in range(len(particles)):
                #b = particles[beta]
                M_s_ab = m_s[alpha] + m_s[beta]
                M_ab = m[alpha] + m[beta]
                '''Add initial term for E_a_b'''
                if beta == alpha:   
                    HFMatrix[alpha][beta] += singleparticleH[alpha]
                    spenergies, CMatrix = np.linalg.eigh(HFMatrix)
                DensityMatrix = np.zeros([spOrbitals,spOrbitals])

                for gamma in range(len(particles)):
                    #c = particles[gamma]

                    for delta in range(len(particles)):
                        #d = particles[delta]
                        M_s_cd = m_s[gamma] + m_s[delta]
                        M_cd = m[gamma] + m_s[delta]
                        C_sum = 0
                        direct_exchange_term = 0.0

                        '''Test for spin and M conservation'''
                        if M_s_ab == M_s_cd and M_ab == M_cd:
                            direct = two_interaction[alpha][gamma]*two_interaction[beta][delta]
                            exchange = two_interaction[alpha][delta]*two_interaction[beta][gamma]
                            '''Coulomb '''
                            '''Direct *- Coulomb(alpha,beta,gamma,delta)'''
                            #direct *= CoulombMatrix[alpha][beta][gamma][delta]
                            direct_exchange_term = (direct - exchange)/2
                            #direct_exchange_term *= 


                        '''Summing C terms'''
                        for j in range(NParticles):
                            C_sum += CMatrix[j][gamma]*CMatrix[j][delta]

                        '''Update Density and HF Matrix'''
                        DensityMatrix[gamma][delta] = Decimal(C_sum)# + Decimal(direct_exchange_term)
                        HFMatrix[alpha][beta] += C_sum*direct_exchange_term
                
                newenergies = spenergies
                """ Brute force computation of difference between previous and new sp HF energies """
                sum_ =0.0
                for i in range(spOrbitals):
                    sum_ += (abs(newenergies[i]-oldenergies[i]))/spOrbitals
                difference = sum_
                oldenergies = newenergies
                


                        #print("<{},{}|{},{}> = ({}*{})*({}*{}) = {}".format(a,b,c,d,a.spin_state, c.spin_state, b.spin_state, 
                        #                                                   d.spin_state, a.spin_state.dot(c.spin_state)*b.spin_state.dot(d.spin_state)))
            #HFmatrix[alpha][beta] = Decimal(sumFockTerm
        hf_count += 1
    #hffile.write(newenergies)
print("Final Hartree Fock Matrix \n {}".format(HFMatrix))
print("Final C Matrix \n {}".format(CMatrix))
#print("Final Density Matrix \n {}".format(DensityMatrix))
print("SP energies \n {}".format(spenergies))


Final Hartree Fock Matrix 
 [[  1.51160438e+03  -1.26298916e-15  -4.98310923e+01  -5.82050752e-16
    4.49702698e-04   6.90826834e-16  -3.05542470e+01  -9.07123156e-15
   -5.03956065e+01  -1.21969005e-15   0.00000000e+00   1.34449522e-17]
 [ -1.27880703e-15   3.07204610e+03  -1.03084454e-14   2.27694070e-04
    4.60365538e-15  -5.02140566e+01  -1.42997365e-17   0.00000000e+00
   -2.39386740e-15  -4.99538986e+01  -1.13836642e-15  -4.44250215e+01]
 [ -4.98524841e+01  -1.16512803e-16   4.67594482e+03  -8.11621237e-15
   -5.03956679e+01   3.38529195e-15  -3.27076949e-01  -9.43932853e-17
   -4.98292922e+01   4.62784637e-16   4.13536577e-04  -3.29117118e-17]
 [ -1.20138691e-14  -1.99854376e-05  -1.56986279e-16   6.24000000e+03
   -1.27067074e-15  -4.99591257e+01  -4.50315825e-17   0.00000000e+00
   -1.37619072e-14  -1.25678592e-04   3.51556812e-15  -5.02490045e+01]
 [  4.54971371e-04  -2.38910913e-16  -5.03956199e+01  -1.93137518e-15
    7.80000000e+03   4.22797831e-16  -4.98620326e+01   7.6

In [ ]:
'''Testing access to two_body matrix'''
with open("nucleitwobody.dat", "r") as twobodyfile:
    for line in twobodyfile:
        nums = line.split(",")
        two_interaction[int(nums[0])][int(nums[1])] = int(nums[2])
        
        
for alpha in range(len(particles)):
        for beta in range(len(particles)):
            a = particles[alpha]
            b = particles[beta]
            M_s_ab = a.m_s + b.m_s
            M_ab = a.m + b.m
            '''Add initial term for E_a_b'''
            if beta == alpha:   
                HFMatrix[alpha][alpha] += singleparticleH[alpha]
                spenergies, CMatrix = np.linalg.eigh(HFMatrix)
            #print("<{}|{}> = {}*{} = {}".format(a.state,b.state, a.spin_state, b.spin_state, a.spin_state.dot(b.spin_state)))
            for gamma in range(len(particles)):
                c = particles[gamma]

                for delta in range(len(particles)):
                    d = particles[gamma]

print(two_interaction)

In [ ]:
sum_ = 0.0
for i in range(Nparticles):
    sum_ += C[gamma][i]*C[delta][i]
    print("gamma: {} i: {} delta: {}".format(gamma,i,delta))
DensityMatrix[gamma][delta] = Decimal(sum_)

In [ ]:
# #Fill the table
# Old Code
# import pandas as pd

# def fill_arrays(n):
#     for i in range(n-1):
#         nx_ny_lst = nx_ny(i)
#         n_m_lst = n_m(i+1)
#         print(nx_ny_lst)
#         print(n_m_lst)
#         for j in range(i+1):
#             particle = single_particle(i+1)
#             particle.nx_ny = nx_ny_lst[j]
#             particle.n_m = n_m_lst[j]

#             particles.append(particle)
#             data = particle.data_tuple()
#             shell_arr[i] = data[0]
#             degeneracy_arr[i] = data[1]
#             E_nx_ny_arr[i] = data[2]
#             magic_number_arr[i] = data[3]
#             energy_arr[i] = data[4]
#             nx_ny_arr.append(data[5])
#             n_m_arr.append(data[6])
#             wavefunction_arr.append(data[7])

# #attempt to generalize fill_arrays rather than hard code
# def fill_arrays_2(n, quantities):
#     for i in range(1,n):
#         particle = single_particle(i)
#         particles.append(particle)
#         data = (particle.shell, particle.degeneracy, particle.E_nx_ny, particle.magic_number, 
#                 particle.energy, particle.nx_ny, particle.n_m, particle.wavefunction)
        
#         for j in range(len(quantities)-1):
#             print(quantities[j],quantities[j][i-1],data[j])
#             if type(quantities) == 'list':
#                 quantities[j].append(data[j])
#             else:
#                 quantities[j][i-1] = data[j]

# n=10
# data_dict = {}

# shell_arr = np.empty(n-1)
# degeneracy_arr = np.zeros(n-1)
# E_nx_ny_arr = np.zeros(n-1)
# magic_number_arr = np.zeros(n-1)
# energy_arr = np.zeros(n-1) 
# nx_ny_arr = []
# n_m_arr = []
# wavefunction_arr = []
# particles = []
# quantities = [shell_arr, degeneracy_arr, E_nx_ny_arr, magic_number_arr, energy_arr, nx_ny_arr, n_m_arr, wavefunction_arr]
# names = ("Shell", "Degeneracy", "E_nx_ny", "Magic Numbers", "Energy", "(nx, ny)", "(n, m)", "Wavefunctions")

# fill_arrays(n)

# #fill_arrays_2(n,quantities)

# for i in range(len(names)):
#     data_dict[names[i]] = quantities[i]
# pd.DataFrame(data_dict)
